In [1]:
#All library imports
import os
import numpy as np
import torch
import numbers
import ntpath
import math
import piq
import lpips
import functools
from PIL import Image
from torchvision import transforms, utils
import cv2
from torch.utils import data
from skimage import io, transform
import pandas as pd
import json
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as TF

import natsort, glob, pickle, torch
from collections import OrderedDict
from utils.util import opt_get
import options.options as option
from models import create_model
from imresize import imresize
import Measure

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#Important function definitions

def find_files(wildcard): return natsort.natsorted(glob.glob(wildcard, recursive=True))

def imshow(array):
    display(Image.fromarray(array))

#from test import load_model, fiFindByWildcard, imread

def pickleRead(path):
    with open(path, 'rb') as f:
        return pickle.load(f)

def load_model(conf_path):
    opt = option.parse(conf_path, is_train=False)
    opt['gpu_ids'] = None
    opt = option.dict_to_nonedict(opt)
    model = create_model(opt)

    model_path = opt_get(opt, ['model_path'], None)
    model.load_network(load_path=model_path, network=model.netG)
    return model, opt

def fiFindByWildcard(wildcard):
    return natsort.natsorted(glob.glob(wildcard, recursive=True))

def imread(path):
    return cv2.imread(path)[:, :, [2, 1, 0]]

# Convert to tensor
def t(array): return torch.Tensor(np.expand_dims(array.transpose([2, 0, 1]), axis=0).astype(np.float32)).to('cuda') / 255

# convert to image
def rgb(t): return (np.clip((t[0] if len(t.shape) == 4 else t).detach().cpu().numpy().transpose([1, 2, 0]), 0, 1) * 255).astype(np.uint8)

# calculates mean, std, min, max values for IQA metrics
def metric_stats(metric_dict_list):
    metric_dict_arr = np.array(metric_dict_list)
    stats_min = round(np.min(metric_dict_arr), 3)
    stats_max = round(np.max(metric_dict_arr), 3)
    stats_mean = round(np.mean(metric_dict_arr), 3)
    stats_std = round(np.std(metric_dict_arr), 3)
    
    return [stats_min, stats_max, stats_mean, stats_std]

In [3]:
find_files("../pretrained_models/*.pth")

['../pretrained_models/ESRGAN_DF2K_8X.pth',
 '../pretrained_models/RRDB_CelebA_8X.pth',
 '../pretrained_models/RRDB_DF2K_4X.pth',
 '../pretrained_models/RRDB_DF2K_8X.pth',
 '../pretrained_models/SRFlow_CelebA_8X.pth',
 '../pretrained_models/SRFlow_DF2K_4X.pth',
 '../pretrained_models/SRFlow_DF2K_8X.pth']

In [4]:
find_files("confs/*.yml")

['confs/RRDB_CelebA_8X.yml',
 'confs/RRDB_DF2K_4X.yml',
 'confs/RRDB_DF2K_8X.yml',
 'confs/SRFlow_CelebA_8X.yml',
 'confs/SRFlow_DF2K_4X.yml',
 'confs/SRFlow_DF2K_8X.yml']

In [5]:
#choose the pre-trained SRFlow model yml file in 'conf_path' variable
conf_path = 'confs/SRFlow_CelebA_8X.yml'
conf_path2='confs/SRFlow_DF2K_4X.yml'
modelX8, opt1 = load_model(conf_path)
modelX4,opt2=load_model(conf_path2)

OrderedDict([('manual_seed', 10), ('lr_G', 0.0005), ('weight_decay_G', 0), ('beta1', 0.9), ('beta2', 0.99), ('lr_scheme', 'MultiStepLR'), ('warmup_iter', -1), ('lr_steps_rel', [0.5, 0.75, 0.9, 0.95]), ('lr_gamma', 0.5), ('niter', 200000), ('val_freq', 40000), ('lr_steps', [100000, 150000, 180000, 190000])])
32
32
0.5
4
16
[1, 3, 5, 7]
[1, 3, 5, 7]
True
True
True
True
True
auto
../pretrained_models/SRFlow_CelebA_8X.pth
OrderedDict([('manual_seed', 10), ('lr_G', 0.00025), ('weight_decay_G', 0), ('beta1', 0.9), ('beta2', 0.99), ('lr_scheme', 'MultiStepLR'), ('warmup_iter', -1), ('lr_steps_rel', [0.5, 0.75, 0.9, 0.95]), ('lr_gamma', 0.5), ('niter', 200000), ('val_freq', 40000), ('lr_steps', [100000, 150000, 180000, 190000])])
32
32
0.5
3
16
[1, 8, 15, 22]
[1, 8, 15, 22]
True
True
True
True
auto
../pretrained_models/SRFlow_DF2K_4X.pth


In [16]:
#initialize LPIPS class
def print_val(variable_name,vals_):
    # variable_name = [name for name, value in globals().items() if value is vals_][0]
    # print(f"Variable name using globals(): {variable_name}")
    vals=np.array(vals_)
    print(variable_name,"-- min:",str(vals.min())," max:",str(vals.max())," avg:",str(vals.mean()))
    
def IDA_RD(lr_dir,hr_dir,num_imgs):
    lq = imread(lr_dir)
    gt = imread(hr_dir)
        #print(lq_paths[i])
        
    lq_size=lq.shape[0]
    hq_size=gt.shape[0]

    gt = torch.tensor(gt).permute(2, 0, 1)[None, ...] / 255
    #for temperature in np.linspace(0, 1, num=11):
    lpips_vals, ssim_vals = [], []
    ms_ssim_vals, psnr_vals = [], []
    
    for j in range(num_imgs):
    
    # Sample a super-resolution for a low-resolution image
        leve_size=hq_size/lq_size
        # print(leve_size)
        if leve_size==4:
            # print(leve_size)
            model=modelX4
            sr = rgb(model.get_sr(lq=t(lq), heat=1.1))
        elif leve_size==8:
            model=modelX8
            sr = rgb(model.get_sr(lq=t(lq), heat=1))
        
    
        if leve_size ==32:
            #print(1)
            sr1 = rgb(modelX8.get_sr(lq=t(lq), heat=1))
            sr=rgb(modelX4.get_sr(lq=t(sr1), heat=0.1))
       
        sr = torch.tensor(sr).permute(2, 0, 1)[None, ...] / 255.
        # print((gt).shape, sr.shape)
        gt = gt.cuda()
        sr = sr.cuda()
    
        ssim_index = piq.ssim(gt, sr, data_range=1.)
        psnr_index = piq.psnr(gt, sr, data_range=1., reduction='none')
        ms_ssim_index = piq.multi_scale_ssim(gt, sr, data_range=1.)
        lpips_index = loss_fn_alex(2*gt-1, 2*sr-1)
    
        ssim_vals.append(ssim_index.item())
        psnr_vals.append(psnr_index.item())
        lpips_vals.append(lpips_index.item())
        ms_ssim_vals.append(ms_ssim_index.item())
    
    # print_val('SSIM',ssim_vals)
    # print_val('PSNR',psnr_vals)
    print_val('LPIPS',lpips_vals)
    # print_val('ms-SSIM',ms_ssim_vals)
    
loss_fn_alex = lpips.LPIPS(net='alex') # best forward scores
if torch.cuda.is_available():
    loss_fn_alex.cuda()


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /home/yuanbangliang/anaconda3/envs/p_r_hubs/lib/python3.8/site-packages/lpips/weights/v0.1/alex.pth


In [8]:
print(modelX8)

In [17]:
from tqdm import tqdm_notebook

hr_dir = 'testing_images/ori_image/0898.png' 
num_imgs = 5 #number of unique HR images to compute for 1 LR image
lr_dir = 'testing_images/lr_image/BICUBIC_256/0898.png'
IDA_RD(lr_dir, hr_dir,num_imgs)

True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True


/tmp/ipykernel_118747/53249895.py:34: RuntimeWarning: invalid value encountered in cast
  def rgb(t): return (np.clip((t[0] if len(t.shape) == 4 else t).detach().cpu().numpy().transpose([1, 2, 0]), 0, 1) * 255).astype(np.uint8)


True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
LPIPS -- min: 0.059829387813806534  max: 0.09941761940717697  avg: 0.07030895948410035


In [18]:
lr_dir = 'testing_images/lr_image/BILINEAR_256/0898.png'
IDA_RD(lr_dir, hr_dir,num_imgs)

True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
LPIPS -- min: 0.042730726301670074  max: 0.0452091284096241  avg: 0.04426551535725594


In [19]:
lr_dir = 'testing_images/lr_image/NEAREST_256/0898.png'
IDA_RD(lr_dir, hr_dir,num_imgs)

True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True


/tmp/ipykernel_118747/53249895.py:34: RuntimeWarning: invalid value encountered in cast
  def rgb(t): return (np.clip((t[0] if len(t.shape) == 4 else t).detach().cpu().numpy().transpose([1, 2, 0]), 0, 1) * 255).astype(np.uint8)


True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
True
[1, 8, 15, 22]
True
[1, 8, 15, 22]
True
LPIPS -- min: 0.716343343257904  max: 0.7446193099021912  avg: 0.7297610640525818
